In [1]:
import sys
sys.path[0]+='/..'
# from graph_dataset.dataset import GraphDataset
from rgcn_dataset import GraphDataset

from utilities import *
import torch
import torch_geometric
import os 
import time
from readdataUC import UC
import gurobipy as gp
import json
from torch_geometric.data import HeteroData
from torch_geometric.nn import RGCNConv
import copy


In [2]:
from torch_geometric.nn import GCNConv

class RGCN(torch.nn.Module):
    def __init__(self):
        super(RGCN, self).__init__()
        self.node_types=['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']
        self.init_sizes=[6,6,6,4,4,4,4,4,4,4]
        in_channels = 4
        hidden_channels = 4
        out_channels = 4

        num_relations = 3*7
        # self.conv1 = RGCNConv(in_channels, hidden_channels,
        #                       num_relations=num_relations, num_bases=1)
        # self.conv2 = RGCNConv(hidden_channels, out_channels,
        #                       num_relations=num_relations, num_bases=1)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)


        self.lins = torch.nn.ModuleList()
        
        for i in range(len(self.node_types)):
            lin = torch.nn.Sequential(
                torch.nn.Linear(self.init_sizes[i], in_channels),
                )
            self.lins.append(lin)

        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(out_channels, out_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(out_channels, 1, bias=False)
        )

    def trans_dimensions(self, g):
        data = copy.deepcopy(g)
        for node_type, lin in zip(self.node_types, self.lins):
            data[node_type].x = lin(data[node_type].x)

        return data
    
    

    def forward(self, data, n):
        data = self.trans_dimensions(data)
        homogeneous_data = data.to_homogeneous()
        edge_index, edge_type = homogeneous_data.edge_index, homogeneous_data.edge_type  
        x = homogeneous_data.x
        # x = self.conv1(x, edge_index, edge_type)
        # x = self.conv2(x, edge_index, edge_type)
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = x[:n]
        output = self.output_module(x).sigmoid()
        return output

In [3]:
LEARNING_RATE = 1e-3
NB_EPOCHS =1000
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-5

In [4]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE = 'cpu'
PredictModel = RGCN().to(DEVICE)



In [5]:
instance = '80_c11_based_8_std'
pt_path = f'../datasets/{instance}/1bin/valid/'
files = os.listdir(pt_path)[:1]



# data = torch.load(pt_path)
# graph = data['graph'].to(DEVICE)
# weighted_sol = data['weighted_sol'].to(DEVICE)

In [7]:
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=0.1)
PredictModel.train()
for i in range(5):
    mean_loss=[]
    for file in files:
        path = os.path.join(pt_path,file)
        data = torch.load(path)
        graph = data['graph'].to(DEVICE)
        weighted_sol = data['weighted_sol'].to(DEVICE)

        sol = PredictModel(graph, 1440).sigmoid()
        weighted_sol = weighted_sol.reshape(1440,1)
    

        loss = loss_fn(sol, weighted_sol)
        optimizer.zero_grad()
        loss.backward()

        # for name, param in PredictModel.named_parameters():
        #     if param.requires_grad:
        #         print(name, param.grad)

        optimizer.step()
        mean_loss.append(loss.item())
    print(sum(mean_loss)/len(mean_loss))
# print(loss)


0.6931471824645996
0.6931471824645996
0.6931471824645996
0.6931471824645996
0.6931471824645996
